<a href="https://colab.research.google.com/github/jodiperwira/Phase-Picking-Using-PhaseNet-Methode/blob/main/PhaseNet%20on%20Dataset%20Indo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seisbench
!pip install obspy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import obspy
from obspy.core import UTCDateTime
import joblib
from keras import models
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import seisbench
import seisbench.generate as sbg
import seisbench.models as sbm
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import random

In [ ]:
list_values = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Interpolate/Data2/Hasil3(fix)_Jatim_100Hz.npy')
list_pphase = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Interpolate/Data2/plabel(fix)_100Hz.npy')
list_sphase = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Interpolate/Data2/slabel(fix)_100Hz.npy')
error_indices = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Interpolate/Data2/error_indices(fix)_.npy')
print(list_values.shape)
print(list_pphase.shape)
print(list_sphase.shape)

In [ ]:
data = np.delete(list_values, error_indices, axis=0)

data.shape

In [ ]:
np.count_nonzero(np.isnan(data))

In [ ]:
# Inisialisasi list untuk menyimpan "slices" unik
unique_slices = [data[0]]
deleted_indices = []

# Iterasi melalui array dan tambahkan "slice" yang unik ke list
for i in range(1, data.shape[0]):
    if not np.array_equal(data[i], data[i-1]):
        unique_slices.append(data[i])
    else:
        deleted_indices.append(i-1)

# Ubah list kembali ke array 3D
cleaned_data = np.array(unique_slices)

In [ ]:
cleaned_plabel = np.delete(list_pphase, deleted_indices, axis=0)
cleaned_slabel = np.delete(list_sphase, deleted_indices, axis=0)

In [ ]:
print("data: ", cleaned_data.shape)
print("p label: ", cleaned_plabel.shape)
print("s label: ", cleaned_slabel.shape)

In [ ]:
plt.figure(figsize=(20,8))
# Combine E, N, and Z in one subplot
plt.subplot(211)
plt.plot(data[892][:,0], label="E")
plt.plot(data[892][:,1], label="N")
plt.plot(data[892][:,2], label="Z")
plt.legend(loc='upper left')

# Plot P-confirmed and S-confirmed
plt.subplot(212)
plt.title("Grafik Analisa Pengujian Data Asli terhadap Hasil Prediksi", fontsize=10)
plt.xlabel("Time", fontsize=10)
plt.ylabel("Probabilitas", fontsize=10)
plt.plot(list_pphase[892], label="P-confirmed")
plt.plot(list_sphase[892], label="S-confirmed")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
arr_reshaped = np.moveaxis(cleaned_data, -1, -2)

arr_reshaped.shape

In [ ]:
poin_p = np.where(cleaned_plabel == 1)
max_data = np.zeros([arr_reshaped.shape[0]])
min_data = np.zeros([arr_reshaped.shape[0]])

for i, v in zip(poin_p[0], poin_p[1]):
    max_data[i] = arr_reshaped[i, :, v:].max(0).max(0)
    min_data[i] = arr_reshaped[i, :, v:].min(0).min(0)

# Ganti seluruh nilai yang lebih besar dari max_data pada setiap indeks dan channel data dengan 0
for i in range(arr_reshaped.shape[0]):
    for j in range(arr_reshaped.shape[1]):
        for z in range(arr_reshaped.shape[2]):
            if arr_reshaped[i, j, z] > max_data[i]:
                arr_reshaped[i, j, z] = 0
            if arr_reshaped[i, j, z] < 0:
                if arr_reshaped[i, j, z] < min_data[i]:
                    arr_reshaped[i, j, z] = 0

In [ ]:
print("data: ", arr_reshaped.shape)
print("p label: ", cleaned_plabel.shape)
print("s label: ", cleaned_slabel.shape)

In [ ]:
plt.figure(figsize=(15,5))

# Combine E, N, and Z in one subplot
plt.subplot(211)
plt.plot(arr_reshaped[892][0,:], label="E")
plt.plot(arr_reshaped[892][1,:], label="N")
plt.plot(arr_reshaped[892][2,:], label="Z")
plt.legend(loc='upper left')

# Plot P-confirmed and S-confirmed
plt.subplot(212)
plt.plot(cleaned_plabel[892], label="P-confirmed")
plt.plot(cleaned_slabel[892], label="S-confirmed")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
combined_labels = np.stack((cleaned_plabel, cleaned_slabel))
combined_labels.shape

In [ ]:
def gaussian(x, mu=0, sigma=10):
    return np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))

sigma = 10
data_sum = combined_labels[0].shape[0]

prob_labels = np.zeros_like(combined_labels, dtype=np.float32)

# Untuk label P (indeks 0)
for i in range(data_sum):
    for j in range(3000):  # Change 3001 to 3000 to stay within array bounds
        if combined_labels[0, i, j] == 1:
            for offset in range(-3*sigma, 3*sigma):  # 3*sigma sebagai jangkauan untuk mendapatkan sebagian besar efek Gaussian
                if 0 <= j + offset < 3000:  # Change 3001 to 3000 to stay within array bounds
                    prob_labels[0, i, j+offset] += gaussian(offset)

# Sama untuk label S (indeks 1)
for i in range(data_sum):
    for j in range(3000):  # 3000
        if combined_labels[1, i, j] == 1:
            for offset in range(-3*sigma, 3*sigma):
                if 0 <= j + offset < 3000:  # 3000
                    prob_labels[1, i, j+offset] += gaussian(offset)

# Normalisasi jika diperlukan
prob_labels = np.clip(prob_labels, 0, 1)

In [ ]:
prob_noise = np.ones_like(prob_labels[0], dtype=np.float32)

for i in range(data_sum):
    for j in range(3000):  # Assuming the range is up to 3000 based on your code
        prob_noise[i, j] = 1 - prob_labels[0, i, j] - prob_labels[1, i, j]

prob_noise = np.clip(prob_noise, 0, 1)

plt.plot(prob_noise[0])

In [ ]:
prob_noise_expanded = prob_noise[np.newaxis, :, :]
final_labels = np.concatenate((prob_noise_expanded, prob_labels), axis=0)

final_labels[0][0]

In [ ]:
plt.figure(figsize=(15,8))

# Combine E, N, and Z in one subplot
plt.subplot(211)
plt.plot(arr_reshaped[892][0,:], label="E")
plt.plot(arr_reshaped[892][1,:], label="N")
plt.plot(arr_reshaped[892][2,:], label="Z")
plt.legend(loc='upper left')

# Plot P-confirmed and S-confirmed
plt.subplot(212)
plt.title("Label", fontsize=10)
plt.xlabel("Time", fontsize=10)
plt.ylabel("Amplitude", fontsize=10)
# plt.plot(final_labels[0][892], label="N-confirmed")
plt.plot(final_labels[1][892], label="P-confirmed")
plt.plot(final_labels[2][892], label="S-confirmed")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
np.save('/content/drive/MyDrive/Jodi/Data2/fian_clean_labels_100Hz.npy', prob_labels)
np.save('/content/drive/MyDrive/Jodi/Data2/fian_clean_data_100Hz.npy', arr_reshaped)

In [ ]:
prob_labels = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Model/clean_labels_100Hz_sigma10.npy')
arr_reshaped = np.load('/content/drive/MyDrive/PhaseNet/Hasil/Model/clean_data_100Hz_sigma10.npy')

In [ ]:
def standard_deviation_normalization(data):
    mean = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean) / std_dev
    return normalized_data

std_deviation_data = standard_deviation_normalization(arr_reshaped)

std_deviation_data

In [ ]:
def robust_scaling(data):
    median = np.median(data)
    iqr = np.percentile(data, 75) - np.percentile(data, 25)

    robust_scaled_data = (data - median) / iqr

    return robust_scaled_data

robust_data = robust_scaling(arr_reshaped)

robust_data

In [ ]:
def max_abs_scaling(data):
    max_abs = np.max(np.abs(data))

    if max_abs == 0:
        # Avoid division by zero
        return data
    else:
        max_abs_scaled_data = data / max_abs
        return max_abs_scaled_data

max_abs_data = max_abs_scaling(arr_reshaped)

max_abs_data

In [ ]:
plt.figure(figsize=(15,5))

# Combine E, N, and Z in one subplot
plt.subplot(111)
plt.plot(std_deviation_data[0][0,:], label="E")
plt.plot(std_deviation_data[0][1,:], label="N")
plt.plot(std_deviation_data[0][2,:], label="Z")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

# Combine E, N, and Z in one subplot
plt.subplot(111)
plt.plot(max_abs_data[0][0,:], label="E")
plt.plot(max_abs_data[0][1,:], label="N")
plt.plot(max_abs_data[0][2,:], label="Z")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

# Combine E, N, and Z in one subplot
plt.subplot(111)
plt.plot(robust_data[0][0,:], label="E")
plt.plot(robust_data[0][1,:], label="N")
plt.plot(robust_data[0][2,:], label="Z")
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return self.data.shape[0]  # mengembalikan jumlah total sampel

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[:, idx]  # ambil data dari dua label P dan S

        return x, y

In [ ]:
total = std_deviation_data.shape[0]

train = int(0.8 * total)
val = int(0.1 * total)
test = int(total - train - val)

x_train = std_deviation_data[:train]
y_train = final_labels[:,:train]

x_val = std_deviation_data[train:train + val]
y_val = final_labels[:,train:train + val]

x_test = std_deviation_data[train + val:]
y_test = final_labels[:,train + val:]

train_size = x_train.shape[0]
val_size = x_val.shape[0]
test_size = x_test.shape[0]

print("Jumlah data set train: ", train_size)
print("Jumlah data set val: ", val_size)
print("Jumlah data set test: ", test_size)

In [ ]:
train_generator = CustomDataset(x_train, y_train)
dev_generator = CustomDataset(x_val, y_val)
test_generator = CustomDataset(x_test, y_test)

In [ ]:
train_loader = DataLoader(train_generator, batch_size=64, shuffle=True)
val_loader = DataLoader(dev_generator, batch_size=64, shuffle=False)

In [ ]:
model = sbm.PhaseNet.from_pretrained("original")

In [ ]:
model = sbm.PhaseNet(phases="PSN",norm="peak")
model

In [ ]:
learning_rate = 1e-3
epochs = 20

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def loss_fn(y_pred, y_true, eps=1e-5):
    # vector cross entropy loss
    h = y_true * torch.log(y_pred + eps)
    h = h.mean(-1).sum(-1)  # Mean along sample dimension and sum along pick dimension
    h = h.mean()  # Mean over batch axis
    return -h

In [ ]:
def f1(y_true, y_pred, threshold):
    def recall(y_true, y_pred, threshold):
        true_positives = torch.sum((y_true * y_pred).float())
        possible_positives = torch.sum(y_true.float() > threshold)
        recall = true_positives / (possible_positives + torch.finfo(torch.float32).eps)
        return recall

    def precision(y_true, y_pred, threshold):
        true_positives = torch.sum((y_true * y_pred).float())
        predicted_positives = torch.sum(y_pred.float() > threshold)
        precision = true_positives / (predicted_positives + torch.finfo(torch.float32).eps)
        return precision

    precision_value = precision(y_true, y_pred, threshold)
    recall_value = recall(y_true, y_pred, threshold)
    f1_value = 2 * ((precision_value * recall_value) / (precision_value + recall_value + torch.finfo(torch.float32).eps))

    return f1_value.item()


In [ ]:
import torch

def mae(y_true, y_pred, tolerance):
    mae_total = 0
    count = 0  # To keep track of instances where error > tolerance

    for i in range(y_true.shape[0]):
        if 1 in y_true[i]:
            y_true_idx = torch.argmax(y_true[i], dim=0).float()
            y_pred_idx = torch.argmax(y_pred[i], dim=0).float()
            error = torch.abs(y_true_idx - y_pred_idx)

            # Only include errors that are above the tolerance level
            if error >= tolerance:
                mae_total += error
                count += 1  # Count this instance for averaging

    # Calculate average MAE for instances where error > tolerance
    if count > 0:  # Ensure we're not dividing by zero
        mae_avg = mae_total / count
        return mae_avg.item() / 100  # Assuming you want to keep the division by 100
    else:
        mae_avg = 0
        return mae_avg


In [ ]:
# Define the threshold value
threshold = 0.5

def train_loop(dataloader):
    size = len(dataloader.dataset)
    for batch_id, (data_batch, label_batch) in enumerate(dataloader):
        data_batch, label_batch = data_batch.float().to(model.device), label_batch.float().to(model.device)

        # Compute prediction and loss
        pred = model(data_batch.to(model.device))
        loss = loss_fn(pred, label_batch.to(model.device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_binary = pred.detach().float()
        f1_p = f1(label_batch[:, 1, :], pred_binary[:, 1, :], threshold)
        f1_s = f1(label_batch[:, 2, :], pred_binary[:, 2, :], threshold)

        mae_p = mae(label_batch[:, 1, :], pred_binary[:, 1, :], threshold)
        mae_s = mae(label_batch[:, 2, :], pred_binary[:, 2, :], threshold)

        if batch_id % 5 == 0:
            loss, current = loss.item(), batch_id * data_batch.shape[0]
            print(f"loss: {loss:>7f}, F1 Score P: {f1_p:>7f}, F1 Score S: {f1_s:>7f}, MAE P: {mae_p:>7f}, MAE S: {mae_s:>7f}  [{current:>5d}/{size:>5d}]")

    return loss, f1_p, f1_s, mae_p, mae_s

def test_loop(dataloader):
    num_batches = len(dataloader)
    val_test_loss = 0
    val_f1_p = 0
    val_f1_s = 0
    val_mae_p = 0
    val_mae_s = 0

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for (data_batch, label_batch) in dataloader:
            data_batch, label_batch = data_batch.float().to(model.device), label_batch.float().to(model.device)
            pred = model(data_batch.to(model.device))
            val_test_loss += loss_fn(pred, label_batch.to(model.device)).item()
            pred_binary = pred.detach().float()
            val_f1_p += f1(label_batch[:, 1, :], pred_binary[:, 1, :], threshold)
            val_f1_s += f1(label_batch[:, 2, :], pred_binary[:, 2, :], threshold)
            val_mae_p += mae(label_batch[:, 1, :], pred_binary[:, 1, :], threshold)
            val_mae_s += mae(label_batch[:, 2, :], pred_binary[:, 2, :], threshold)

    model.train()  # Set the model back to training mode

    val_test_loss /= num_batches
    val_f1_p /= num_batches
    val_f1_s /= num_batches
    val_mae_p /= num_batches
    val_mae_s /= num_batches
    print(f"Val avg loss: {val_test_loss:>8f}, Val avg F1-P: {val_f1_p:>8f}, Val avg F1-S: {val_f1_s:>8f}, Val avg MAE-P: {val_mae_p:>8f}, Val avg MAE-S: {val_mae_s:>8f} \n")

    return val_test_loss, val_f1_p, val_f1_s, val_mae_p, val_mae_s


In [ ]:
loss_history = []
f1_scorep_history = []
f1_scores_history = []
mae_p_history = []
mae_s_history = []

# # Set a patience threshold for early stopping
# patience = 12  # Adjust this value as needed

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    # Train loop
    loss, f1_p, f1_s, mae_p, mae_s = train_loop(train_loader)

    # Test loop
    val_test_loss, val_f1_p, val_f1_s, val_mae_p, val_mae_s = test_loop(val_loader)

    # Append results to history lists
    loss_history.append(val_test_loss)
    f1_scorep_history.append(val_f1_p)
    f1_scores_history.append(val_f1_s)
    mae_p_history.append(val_mae_p)
    mae_s_history.append(val_mae_s)

    # # Early stopping logic
    # if t >= patience:  # Ensure enough epochs for initial evaluation
    #     if val_test_loss >= min(loss_history[t-patience:]):  # Compare with minimum loss in patience window
    #         print(f"Early stopping at epoch {t + 1}")
    #         break


In [ ]:
import csv

# Daftar variabel
variables = ['loss_history', 'f1_scorep_history', 'f1_scores_history', 'mae_p_history', 'mae_s_history']

# Daftar nilai variabel
values = [loss_history, f1_scorep_history, f1_scores_history, mae_p_history, mae_s_history]

# Nama file CSV
filename = "training_metrics.csv"

# Menulis ke dalam file CSV
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Menulis header
    writer.writerow(variables)

    # Menulis nilai
    for i in range(len(values[0])):
        row = [values[j][i] for j in range(len(values))]
        writer.writerow(row)


In [ ]:
sample = test_generator[np.random.randint(len(test_generator))]
# Move the tensor to CPU and ensure it is in float format
pred = model(torch.tensor([sample[0]]).float().to(model.device))
y_pred = pred[:,1:].cpu().detach().numpy()

# Rest of your plotting code remains the same
fig = plt.figure(figsize=(15, 10))
axs = fig.subplots(3, 1, sharex=True, gridspec_kw={"hspace": 0, "height_ratios": [3, 1, 1]})
axs[0].plot(sample[0].T)
axs[1].plot(sample[1].T)
# Move the prediction back to CPU if it was on GPU for some reason, and detach it from the current graph
axs[2].plot(y_pred[0].T)

plt.show()  # Don't forget to show the plot if this is the end of your plotting code

In [ ]:
import matplotlib.pyplot as plt

# Misalkan epochs adalah jumlah epochs yang Anda gunakan
epochs = len(loss_history)  # Or the actual number of epochs

# Assuming loss and val_test_loss are already NumPy arrays
loss = loss  # No detach needed
val_test_loss = val_test_loss  # No detach needed

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), loss_history, label='Loss')
plt.plot(range(1, epochs + 1), val_test_loss, label='Validation Loss')
plt.title('Loss selama Training dan Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# # Membuat plot untuk F1 Score
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, epochs + 1), f1_scorep_history, label='F1 Score Positive')
# plt.plot(range(1, epochs + 1), f1_scores_history, label='F1 Score Negative')
# plt.title('F1 Score selama Training dan Validation')
# plt.xlabel('Epochs')
# plt.ylabel('F1 Score')
# plt.legend()
# plt.show()


In [ ]:
x_test = test_generator[:][0]
y_test = test_generator[:][1]

In [ ]:
pred = model(torch.tensor(x_test).float().to(model.device))
pred = pred.cpu().detach().numpy()

In [ ]:
# Assuming y_tes and pred are tensors of shape (batch_size, time_steps, num_classes)

# Menghitung MAE
mae_p = (y_test[1].argmax(1) - pred[:,1,:].argmax(1)) / 100

plt.figure(figsize=(6, 6))
# Ensure the mean is calculated over the correct dimensions and converted properly
plt.text(-0.85, 125, r'$mae=$' + f'{round(abs(y_test[1].argmax(1) - pred[:,1,:].argmax(1)).mean()/100, 5)} s')
plt.xlim(-2, 2)
plt.title('P-Wave')
plt.xlabel('MAE Values (seconds)')
plt.ylabel('Frequency')
plt.xticks(np.linspace(-2, 2, 9))

# Converting the tensor to a numpy array for histogram plotting
plt.hist(mae_p.flatten(), 24, alpha=0.5, edgecolor='black')

plt.show()

In [ ]:
idx = np.where(y_test[2] == 1)[0]

mask = np.isin(np.arange(y_test.shape[1]), idx)

In [ ]:
y_test_s = y_test[:, mask, :]
y_test_s.shape

In [ ]:
mask = np.isin(np.arange(pred.shape[0]), idx)

# Terapkan mask ke pred
pred_s = pred[mask, :, :]
pred_s.shape

In [ ]:
# Assuming y_tes and pred are tensors of shape (batch_size, time_steps, num_classes)

# Menghitung MAE
mae_s = (y_test_s[2].argmax(1) - pred_s[:,2,:].argmax(1)) / 100

plt.figure(figsize=(6, 6))
# Ensure the mean is calculated over the correct dimensions and converted properly
plt.text(-8.85, 125, r'$mae=$' + f'{round(abs(y_test_s[2].argmax(1) - pred_s[:,2,:].argmax(1)).mean()/100, 5)} s')
plt.xlim(-10, 10)
plt.title('S-Wave')
plt.xlabel('MAE Values (seconds)')
plt.ylabel('Frequency')
plt.xticks(np.linspace(-10, 10, 9))

# Converting the tensor to a numpy array for histogram plotting
plt.hist(mae_s.flatten(), 24, alpha=0.5, edgecolor='black')

plt.show()